In [1]:
import pyspark
import findspark
from pyspark.sql import SparkSession

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression

import pandas as pd

from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [2]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

23/04/20 01:22:39 WARN Utils: Your hostname, MacBook-Air-Evgenij.local resolves to a loopback address: 127.0.0.1; using 192.168.0.102 instead (on interface en0)
23/04/20 01:22:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/20 01:22:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
df = pd.read_csv('./santander-customer-transaction-prediction/train.csv')

In [5]:
df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [6]:
df.dtypes

ID_code     object
target       int64
var_0      float64
var_1      float64
var_2      float64
            ...   
var_195    float64
var_196    float64
var_197    float64
var_198    float64
var_199    float64
Length: 202, dtype: object

In [7]:
input_columns = df.columns.tolist()[2:]

In [8]:
dataset = spark.read.csv('./santander-customer-transaction-prediction/train.csv', inferSchema=True, header=True)

In [9]:
input_columns = dataset.columns[2:]

In [10]:
assembler = VectorAssembler(inputCols=input_columns, outputCol='Attributes')

output = assembler.transform(dataset)

finalized_data = output.select('Attributes', 'target')

finalized_data.show()

23/04/20 01:22:46 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+------+
|          Attributes|target|
+--------------------+------+
|[8.9255,-6.7863,1...|     0|
|[11.5006,-4.1473,...|     0|
|[8.6093,-2.7457,1...|     0|
|[11.0604,-2.1518,...|     0|
|[9.8369,-1.4834,1...|     0|
|[11.4763,-2.3182,...|     0|
|[11.8091,-0.0832,...|     0|
|[13.558,-7.9881,1...|     0|
|[16.1071,2.4426,1...|     0|
|[12.5088,1.9743,8...|     0|
|[5.0702,-0.5447,9...|     0|
|[12.7188,-7.975,1...|     0|
|[8.7671,-4.6154,9...|     0|
|[16.3699,1.5934,1...|     1|
|[13.808,5.0514,17...|     0|
|[3.9416,2.6562,13...|     0|
|[5.0615,0.2689,15...|     0|
|[8.4199,-1.8128,8...|     0|
|[4.875,1.2646,11....|     0|
|[4.409,-0.7863,15...|     0|
+--------------------+------+
only showing top 20 rows



In [11]:
train_data, test_data = finalized_data.randomSplit([0.8, 0.2])

regressor = LogisticRegression(featuresCol='Attributes', labelCol='target')

regressor = regressor.fit(train_data)

pred = regressor.evaluate(test_data)

pred.predictions.show()

23/04/20 01:22:51 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/04/20 01:22:51 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
23/04/20 01:22:52 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+--------------------+------+--------------------+--------------------+----------+
|          Attributes|target|       rawPrediction|         probability|prediction|
+--------------------+------+--------------------+--------------------+----------+
|[1.6044,-0.6302,1...|     0|[2.81577733270635...|[0.94352247066797...|       0.0|
|[1.922,-3.31,8.37...|     0|[2.97425341712561...|[0.95139733495655...|       0.0|
|[2.5041,-4.9081,8...|     0|[5.21681693215687...|[0.99460469873667...|       0.0|
|[2.6501,-2.6365,1...|     0|[3.24585880260540...|[0.96252401910992...|       0.0|
|[2.8253,4.2618,15...|     0|[5.54884902547382...|[0.99612315424485...|       0.0|
|[3.6355,4.1567,14...|     1|[1.94706342408893...|[0.87512608491385...|       0.0|
|[3.7917,-1.0115,8...|     0|[4.41978174446677...|[0.98810630363823...|       0.0|
|[3.8296,-7.5727,8...|     0|[3.69750919841062...|[0.97581426332228...|       0.0|
|[3.8649,2.1927,10...|     0|[2.66281427774470...|[0.93479641269697...|       0.0|
|[3.

In [12]:
coef = regressor.coefficients

intr = regressor.intercept

print('coef: %a' % coef)
print('intr: %a' % intr)

coef: DenseVector([0.0531, 0.0386, 0.0591, 0.0221, 0.0224, 0.0147, 0.2342, -0.0038, 0.0217, -0.084, 0.0019, 0.0161, -1.1831, -0.0341, -0.0037, 0.167, 0.0077, -0.0016, 0.0196, 0.0072, -0.011, -0.0235, 0.0629, -0.1602, 0.0346, 0.1419, 0.0388, -0.0081, -0.1076, 0.0057, -0.0006, -0.0439, 0.0314, -0.0396, -0.3118, 0.0262, -0.0395, 0.008, -0.0032, -0.0026, 0.0187, -0.0011, -0.0406, -0.3514, -0.0319, -0.0022, 0.002, 0.0031, 0.0093, 0.0109, -0.0538, 0.0048, 0.0153, 0.2585, -0.0067, 0.0129, -0.0269, -0.0896, -0.0188, -0.0352, 0.0098, 0.0038, 0.0228, -0.0182, -0.0302, 0.01, 0.0784, 0.0206, -5.3815, 0.0062, 0.0056, 0.3775, -0.0163, -0.0008, 0.0059, -0.0266, -0.0236, -0.0115, 0.0718, 0.0341, -0.0283, -0.1025, 0.0099, -0.0064, 0.0099, -0.0256, -0.0193, -0.0147, -0.0225, 0.0341, 0.0061, 0.8587, -0.0326, -0.2586, 0.0663, 0.2246, 0.0026, 0.0058, -0.0157, 0.1014, -0.0009, -0.0107, -0.0068, 0.113, -0.039, 0.1175, 0.065, -0.0161, -0.8179, -0.035, 0.0521, 0.0885, 0.0538, -0.0201, -0.109, -0.0605, -0.0562,

In [13]:
eval = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='target', metricName='areaUnderROC')

In [14]:
roc = eval.evaluate(pred.predictions)
print('ROC: %.3f' % roc)

ROC: 0.853
